In [1]:
pip install openai sounddevice numpy torch transformers pyttsx3 webrtcvad

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/8a/91/1f1cf577f745e956b276a8b1d3d76fa7a6ee0c2b05db3b001b900f2c71db/openai-1.97.0-py3-none-any.whl.metadata
  Obtaining dependency information for sounddevice from https://files.pythonhosted.org/packages/e1/3e/61d88e6b0a7383127cdc779195cb9d83ebcf11d39bc961de5777e457075e/sounddevice-0.5.2-py3-none-win_amd64.whl.metadata
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/50/9e/acf04ff375b0b49a45511c55d188bcea5c942da2aaf293096676110086d1/torch-2.7.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyttsx3 from https://files.pythonhosted.org/packages/84/14/9fb5842581f0419b5eb85f8c26c1c0c0f4cf6b4d5be638ae3157316a2650/pyttsx3-2.99-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/66.2 kB ? eta -:--:--
     ------------------------------------- -- 61.4/66.2 kB 1.7 MB/s eta 0:00:01
     ------------------------------

In [2]:
!apt-get install -y portaudio19-dev python3-pyaudio
!pip install pyaudio sounddevice

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


^C


In [3]:
!pip install langdetect

  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   --------- ----------------------------- 41.0/164.1 kB 960.0 kB/s eta 0:00:01
   ----------------------------- ---------- 122.9/164.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------  163.8/164.1 kB 1.4 MB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 979.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     - ----------------------------------- 41.0/981.5 kB 991.0 kB/s eta 0:00:01
     ---- --------------------------------- 122.9/981.5 kB 1.2 MB/s eta 0:00:01
     -------- ----------------------------- 225.3/981.5 kB 1.5 MB/s eta 0:00:01
     ----------- -------------------------- 286.7/981.5 kB 1.5 MB/s eta 0:00:01
     --

In [4]:
!apt-get install -y portaudio19-dev python3-pyaudio espeak ffmpeg
!pip install sounddevice pyaudio transformers openai gtts webrtcvad librosa pydub

import os
import time
from openai import OpenAI
import numpy as np
from transformers import pipeline
from gtts import gTTS
from IPython.display import Audio, display, Javascript, HTML
import torch
import io
import soundfile as sf
from base64 import b64encode
from pydub import AudioSegment
from langdetect import detect

# Configuration
SAMPLE_RATE = 16000
SAMBASTUDIO_API_KEY = "1dc8dd92-24c8-4760-a1eb-6195af9021a3"  # Replace with your actual key

# Initialize OpenAI client for Sambanova
client = OpenAI(
    base_url="https://api.sambanova.ai/v1",
    api_key=SAMBASTUDIO_API_KEY
)

# Initialize components
print("Initializing speech recognition model...")
speech_recognizer = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    device="cuda" if torch.cuda.is_available() else "cpu"
)
print("Speech recognition model loaded")

def show_recording_interface():
    display(HTML("""
    <div id="recorder-container">
        <button id="startBtn" style="padding: 10px 20px; margin: 5px; background-color: #4CAF50; color: white; border: none; border-radius: 4px;">Start Recording</button>
        <button id="stopBtn" style="padding: 10px 20px; margin: 5px; background-color: #f44336; color: white; border: none; border-radius: 4px;" disabled>Stop Recording</button>
        <div id="status" style="margin: 10px 0; font-weight: bold; color: #333;">Ready to record</div>
    </div>
    """))

    display(Javascript("""
    let recorder, audioStream;
    const startBtn = document.getElementById('startBtn');
    const stopBtn = document.getElementById('stopBtn');
    const statusDiv = document.getElementById('status');

    startBtn.onclick = async () => {
        startBtn.disabled = true;
        stopBtn.disabled = false;
        statusDiv.textContent = 'Recording... Speak now!';
        statusDiv.style.color = '#4CAF50';

        try {
            audioStream = await navigator.mediaDevices.getUserMedia({ audio: true });
            recorder = new MediaRecorder(audioStream);
            const chunks = [];

            recorder.ondataavailable = e => chunks.push(e.data);
            recorder.onstop = async () => {
                const blob = new Blob(chunks, { type: 'audio/webm' });
                const reader = new FileReader();
                reader.onload = () => {
                    const arr = Array.from(new Uint8Array(reader.result));
                    google.colab.kernel.invokeFunction('notebook.process_audio', [arr], {});
                };
                reader.readAsArrayBuffer(blob);
                statusDiv.textContent = 'Processing...';
                statusDiv.style.color = '#FF9800';
            };

            recorder.start();
        } catch (err) {
            console.error('Error:', err);
            statusDiv.textContent = 'Error: ' + err.message;
            statusDiv.style.color = '#f44336';
            startBtn.disabled = false;
        }
    };

    stopBtn.onclick = () => {
        if (recorder && recorder.state !== 'inactive') {
            recorder.stop();
        }
        if (audioStream) {
            audioStream.getTracks().forEach(track => track.stop());
        }
        startBtn.disabled = false;
        stopBtn.disabled = true;
    };
    """))

def process_audio(audio_data):
    """Process recorded audio from Colab"""
    try:
        # Convert to bytes and save as temporary file
        webm_audio = io.BytesIO(bytes(audio_data))
        with open("temp.webm", "wb") as f:
            f.write(webm_audio.getbuffer())

        # Convert WebM to WAV using pydub
        audio = AudioSegment.from_file("temp.webm", format="webm")
        audio = audio.set_frame_rate(SAMPLE_RATE).set_channels(1)
        audio.export("temp.wav", format="wav")

        # Read WAV file
        audio_data, sr = sf.read("temp.wav")

        # Transcribe
        result = speech_recognizer({"raw": audio_data, "sampling_rate": SAMPLE_RATE})
        user_input = result["text"]
        print(f"\nYou said: {user_input}")

        if user_input.strip():
            # Get response from LLM
            response = query_llama(user_input)
            print(f"LLaMA response: {response}")

            # Convert response to speech
            text_to_speech(response)

    except Exception as e:
        print(f"\nError processing speech: {e}")
        text_to_speech("Sorry, I encountered an error processing your request.")

def query_llama(prompt):
    """Send prompt to LLaMA 8B model hosted on Sambanova"""
    try:
        completion = client.chat.completions.create(
            model="Meta-Llama-3.1-8B-Instruct",
            messages=[
                {"role": "system", "content": "Respond concisely to the user's query."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            # max_tokens=150
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error querying Sambanova API: {e}")
        return "I couldn't connect to the AI service. Please try again later."

def detect_language(text):
    """Detect language of the text"""
    try:
        lang = detect(text)
        # Map detected language to gTTS language codes
        lang_map = {
            'en': 'en',    # English
            'hi': 'hi',    # Hindi
            'mr': 'mr',     # Marathi
            # Add more languages as needed
        }
        return lang_map.get(lang, 'en')  # Default to English if unknown
    except:
        return 'en'

def text_to_speech(text):
    """Automatically detect language and convert to speech"""
    # Detect language from the text
    lang = detect_language(text)

    # Create speech synthesis
    tts = gTTS(text=text, lang=lang)
    tts.save("response.mp3")

    # Play the audio
    with open("response.mp3", "rb") as f:
        audio_data = f.read()
    audio_b64 = b64encode(audio_data).decode('utf-8')
    display(Javascript(f"""
    const audio = new Audio('data:audio/mp3;base64,{audio_b64}');
    audio.play();
    """))
    return Audio("response.mp3", autoplay=True)

# Register the processing function with Colab
from google.colab import output
output.register_callback('notebook.process_audio', process_audio)

# Main execution
print("Speech-to-Speech with LLaMA 8B on Sambanova")
show_recording_interface()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


^C
  Obtaining dependency information for gtts from https://files.pythonhosted.org/packages/e3/6c/8b8b1fdcaee7e268536f1bb00183a5894627726b54a9ddc6fc9909888447/gTTS-2.5.4-py3-none-any.whl.metadata
  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/b5/ba/c63c5786dfee4c3417094c4b00966e61e4a63efecee22cb7b4c0387dda83/librosa-0.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydub from https://files.pythonhosted.org/packages/a6/53/d78dc063216e62fc55f6b2eebb447f6a4b0a59f55c8406376f76bf959b08/pydub-0.25.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for soundfile>=0.12.1 from https://files.pythonhosted.org/packages/14/e9/6b761de83277f2f02ded7e7ea6f07828ec78e4b229b80e4ca55dd205b9dc/soundfile-0.13.1-py2.py3-

ImportError: cannot import name 'pipeline' from 'transformers' (c:\Anaconda3\Lib\site-packages\transformers\__init__.py)